In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import InferenceClient
import google.generativeai as genai
import os
import json
from dotenv import load_dotenv


load_dotenv(r"C:\Users\leduc\OneDrive\Desktop\NLP\Grab-project\.env")

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [ ]:
hf_token = os.environ["HF_TOKEN"]

In [3]:
prompt = """
### Instruction:
You are an expert resume and job description parser.

Given the following input text (which may be a resume or job description), extract and structure the relevant information in the following JSON format:

{
    "summary": "Provide a concise professional summary based on the resume.",
    "skills": ["List of skills"],
    "basic_info": {
        "full_name": "Invented Full Name",
        "university": "Generated University Name",
        "education_level": "BS",
        "majors": ["List of Majors", "GPA: 3.5"]
    },
    "work_experience": [{
        "job_title": "Title",
        "company": "Company Name",
        "location": "Location",
        "duration": "Duration",
        "job_summary": "Job responsibilities and achievements"
    }],
    "project_experience": [{
        "project_name": "Project Name",
        "project_description": "Project details including technologies used"
    }],
    "publications": [{
        "publications_name": "Project Name",
        "publications_description": "Project details including technologies used"
    }],
    "award": [{"award_name": "Award Name"}]
}

### Guidelines:
- Summarize the publications and projects description.
- If a field is missing or not available, leave it out of the JSON.
- Do not hallucinate data not implied by the text.
- Return only the JSON object. No explanation or preamble.

Input Text:



"""

In [4]:


def cv_jd_extractor(question):
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": question},
    ]
    client = InferenceClient(
        provider="novita",
        api_key=hf_token,
    )

    completion = client.chat.completions.create(
        model="deepseek-ai/DeepSeek-V3-0324",
        messages=messages,
        max_tokens=1000,
    )
    return completion.choices[0].message.content
    

In [5]:
text = """
  Jackson McCall
Engineer
mca_lljcks12@gmail.com | 781-235-7144 | Norwell,

Profile

Experienced Engineer bringing forth 5+ years providing superior technical guidance and successfully executed engineering designs. Adept in overseeing all aspects of project management including design, implementation, verification, and validation of engineering designs. 

Work Experience

11/2016 - 07/2021, Senior Project Engineer, Valley Field Technologies, Boston

Effectively guided technical project development and helped to steer and improve program processes.
Planned and executed assigned engineering designs with accuracy.
Worked with program managers to ensure proper and adequate technical staffing.
Executed design and development efforts in adherence to all engineering design standards. 
Worked to quickly resolve technical conflicts and issues.
Developed and maintained project timelines.
Monitored and adjusted control systems.

07/2014 - 08/2015, Project Manager, Infotech Builders, Boston

Worked to leverage business acumen, leadership experience, and technical skills to deliver results.
Oversaw all aspects of project budget, planning, design, and installation. 
Assessed and communicated project status and potential risks across multiple teams. 
Planned and conducted weekly standing meetings with internal teams and external vendors and contractors. 
Proposed continuous improvement strategies with the goal of higher deployment rates. 

Education

09/2014 - 05/2016, Master of Science in Systems Engineering, Boston University, Boston

09/2009 - 05/2013, Bachelor of Science in Computer Engineering, Northwestern University, Chicago

Languages
English
German
Skills
PMI Certified Project Management Professional
Microsoft Project
MS Dynamics AX
Advanced Analytical Thinking Skills
Hardware Design
"""

In [6]:
import re

def clean_resume(text):
    text = text.lower()

    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    

    text = re.sub(r'(\+?\d{1,3})?[-.\s]?(\(?\d{3}\)?|\d{3})[-.\s]?\d{3}[-.\s]?\d{4}', '', text)
    

    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    

    text = re.sub(r'\b(Name|Email|Phone|Contact|LinkedIn|Address|Scholar|ORCID)\b[:\s]*', '', text, flags=re.IGNORECASE)
    

    text = re.sub(r'\s+', ' ', text).strip()
    
    return text


In [ ]:
from sentence_transformers import SentenceTransformer, util

import numpy as np

# Load a pre-trained sentence transformer model
model = SentenceTransformer('all-mpnet-base-v2')

# Example resume and JD texts (in practice, you'd load actual texts)
resume_text = """
 
"""

jd_text = """
About the job
RegenX, a start-up in regenerative farming and AI Agvisory, is looking for a 🤖 RAG AI Engineer 🤖 (full time, hybrid work arrangement).


About RegenX:

🌏 We’re on a mission to help smallholder farmers adapt to climate change by providing state-of-the-art AI-agvisory while partnering with the ecosystem to accelerate transitions to regenerative agriculture.


Here is some more specific information about the role.


🎯 The main “Job-To-Be-Done” is:

Build a world class Ai-first platform that enables farmers and buyers to track regenerative transitions progress, outcomes and adjust their actions to adapt to climate change


💼 Skills and background we are looking for:



Proven ability to build, deploy, and optimize multi-modal RAG pipelines (vision, NLP, voice)
Experience with vector databases (Qdrant, Weaviate, Pinecone) and RAG orchestration
Comfortable designing data architectures for scalable, low-latency inference pipelines.
Experience with document parsing and knowledge retrieval, object detection, image preprocessing.
Familiar with cloud-native architectures (GCP, AWS, or Azure), particularly using serverless functions, containerization, and event-driven workflows.
Strong software engineering fundamentals, including testing, CI/CD, and observability.


🔥 Preferred Qualifications:

A solid understanding of software engineering fundamentals coupled with strong AI background 
Experience with PostgreSQL, caching strategies (Redis), and API performance optimization.
Fluent in Python, Typescript / Javascript, Postgresql, caching solutions, edge computing
Familiarity with Supabase, Firebase, or similar backend platforms for fast prototyping
Some understanding of cross-platform development constraints (Flutter, React Native, or platform-specific UIs).
Experience working with vector DBs and image embeddings.
Awareness of data privacy, security, and compliance requirements when dealing with user data and AI systems.


📝 General terms and conditions of the job

Full time
13 months salary, PVI insurance, BHXH based on full month salary, laptop (after probation) and AI tooling, unlimited leaves, hybrid work arrangement
What’s hybrid work? We meet offline twice a week in HCMC (Vietnam), the rest is remote
You may have to travel within Vietnam from time to time to directly meet with farmers and / or partners to build farmer and customer centric tech solutions 
Interview process: A 15-min call to talk about your ambitions → Technical interview → Culture Fit interview. Our hiring team is committed to including diverse perspectives and backgrounds throughout the interview process
Salary range: we’ll discuss based on the assessment during the interview process


"""


resume_text = clean_resume(resume_text)
jd_text = clean_resume(jd_text)

resume_embedding = model.encode(resume_text, convert_to_tensor=True)
jd_embedding = model.encode(jd_text, convert_to_tensor=True)

semantic_similarity = util.pytorch_cos_sim(resume_embedding, jd_embedding).item()


print("Semantic Similarity:", semantic_similarity)



c:\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Semantic Similarity: 0.5775570869445801


In [8]:
genai.configure(api_key=os.environ["GEMINI_TOKEN"])
ge_model = genai.GenerativeModel("gemini-2.0-flash")

In [62]:
def llm_grading(resume_text: str, jd_text: str):
    gemini_input = f"""
    You are an expert technical recruiter and career advisor.

    Given the following job description and candidate resume, do the following:

    1. **Score the overall match from 0 to 1 it is a float number** based on how well the candidate fits the job requirements.
    - if the job title or work is not relevent just return very low score.
    2. **List missing or weak areas**
    3. **Advise on improvements** – suggest what the candidate can add to their CV to better match the role.
    4. **List keywords that missing in resume**
    5. Be concise but clear, and format the response in Markdown for readability.
    6. **OUTPUT FORMAT MUST BE JSON:**

    Respond only in the exact JSON format shown below. Do not add any text outside the JSON block.

    ---

    **Job Description:**
    {jd_text}

    ---

    **Candidate Resume:**
    {resume_text}

    
    **Sample Output Format**
    ```json
    {{
      "overall_match": 0.43,
      "missing_or_weak_areas": [
        "No exposure to Unity, Unreal, or any game engine tools.",
        "No specific focus on AI for game development or visual recognition.",
        "Missing essential programming languages used in games (C#, C++).",
        "Limited evidence of direct involvement in collaborative game or AI projects."
      ],
      "advice_on_improvements": [
        "Start learning Unity or Unreal Engine and build a simple demo.",
        "Pick up a course or tutorial in C++ or C# and implement a basic game mechanic.",
        "Look into AI tasks relevant to game development like enemy pathfinding or procedural world generation.",
        "Engage in team-based AI projects and include your role in collaboration and design."
      ],
      "skills_missing": [
        "Unity",
        "Unreal Engine",
        "C#",
        "C++",
        "Game AI Concepts",
        "Team Collaboration in Game Projects"
      ]
    }}
    ```
    **Sample Output Format**
    {{
      "overall_match": 0.91,
      "missing_or_weak_areas": [],
      "advice_on_improvements": [
        "Continue building advanced AI mechanics such as adaptive enemy behaviors or procedural world systems.",
        "Consider contributing to open-source game AI libraries or plugins for Unity/Unreal.",
        "Explore optimization techniques for real-time AI decision-making under hardware constraints."
      ],
      "skills_missing": []
    }}

    {{
      "overall_match": 0.17,
      "missing_or_weak_areas": [
        "No formal education in pharmacology, chemistry, or medicine.",
        "Lacks required certifications or licensure (e.g., PharmD, pharmacy board exams).",
        "No knowledge of drug interactions, prescriptions, or patient safety protocols.",
        "No clinical experience or understanding of healthcare systems and regulations."
      ],
      "advice_on_improvements": [
        "Enroll in a formal pharmacy program to gain necessary credentials.",
        "Complete internship or training at a certified pharmacy or hospital.",
        "Gain understanding of pharmacokinetics, patient care, and ethical practice.",
        "Prepare for and pass pharmacy licensing exams (e.g., NAPLEX)."
      ],
      "skills_missing": [
        "Pharmacology",
        "Prescription Drug Knowledge",
        "Clinical Experience",
        "Pharmacy Licensure",
        "Healthcare Regulations",
        "Patient Counseling",
        "Medical Ethics"
      ]
    }}



    """

    res = ge_model.generate_content(gemini_input, 
        generation_config={
            "temperature": 0.6,
            "top_k": 2,
            "top_p": 0.9,
            "max_output_tokens": 1000,
        },
        
    )

    grading = json.loads(res.text.replace("```", "").replace("json", ""))
    return grading


In [64]:
s = llm_grading(resume_text, jd_text)
for k, v in s.items():
    print(k, v)
    

overall_match 0.65
missing_or_weak_areas ['The resume does not explicitly state that the candidate is a native-level English speaker from one of the specified countries (USA, UK, Canada, Australia, New Zealand, Ireland, or South Africa).', 'The resume does not explicitly state possession of CELTA/Trinity certification or an MA in TESOL; it only mentions IELTS score.', 'The resume lacks explicit mention of experience teaching adults.', "The resume does not include a cover letter tailored to Wall Street English Vietnam, explaining the candidate's motivation to work there."]
advice_on_improvements ['Clearly state native English speaker status and nationality if applicable.', 'Mention any TEFL certifications held, even if not CELTA/Trinity. Highlight any teaching experience, especially with adults.', "Craft a cover letter emphasizing a passion for teaching, commitment to English education, and reasons for wanting to work at Wall Street English Vietnam. Tailor the cover letter to align with

In [106]:
jd_text = """
About FanduelFanDuel Group is the premier mobile gaming company in the United States and Canada. FanDuel Group consists of a portfolio of leading brands across mobile wagering including: America’s #1 Sportsbook, FanDuel Sportsbook; its leading iGaming platform, FanDuel Casino; the industry’s unquestioned leader in horse racing and advance-deposit wagering, FanDuel Racing; and its daily fantasy sports product.In addition, FanDuel Group operates FanDuel TV, its broadly distributed linear cable television network and FanDuel TV+, its leading direct-to-consumer OTT platform. FanDuel Group has a presence across all 50 states, Canada, and Puerto Rico.The company is based in New York with US offices in Los Angeles, Atlanta, and Jersey City, as well as global offices in Canada and Scotland. The company’s affiliates have offices worldwide, including in Ireland, Portugal, Romania, and Australia.FanDuel Group is a subsidiary of Flutter Entertainment, the world's largest sports betting and gaming operator with a portfolio of globally recognized brands and traded on the New York Stock Exchange (NYSE: FLUT).THE POSITIONOur roster has an opening with your name on itIn addition to the specific responsibilities outlined above, employees may be required to perform other such duties as assigned by the Company. This ensures operational flexibility and allows the Company to meet evolving business needs.Our competitive edge is enabled by the ability to leverage accurate and timely data for informed decision-making. We are seeking a Data Engineer to advance FanDuel’s data capabilities and contribute to the further growth and maturity of our Data Team.As a Data Engineer, you will be a valued member of the Data Support team, maintaining exceptional standards and servicing all the needs of data consumers across the company. Your role will be to ensure the health of our data infrastructure, contribute to the delivery of reliable, timely, and consumable data for all company data consumers. The success of this team will elevate the capabilities of our Data tribe to maximise the value of our data assets and empower employees to innovate.You will have the opportunity to work in an agile environment where you will team up with colleagues having experience in data warehousing, machine learning and more.The successful candidate should have minimum 3-5 years of data engineering experience with good technical and problem-solving skills, a positive and results driven attitude, and a good communicator who can interact with both technical and non-technical people.THE GAME PLANEveryone on our team has a part to playKey Responsibilities:Provide on-call technical assistance to analysts, data consumers, and data engineering teams, responding to inquiries relating to query optimization, reporting anomalies, complex data pipeline issues, failed jobs, and data inconsistencies.Investigate and resolve data quality issues in response to system alerts or notifications from stakeholders, ensuring compliance with internal/external audits and control practices.Design and implement batch and real-time data pipelines to the data warehouse or data lake using data transformation technologies, enhancing monitoring dashboards and support tools, and automating repetitive support tasks.Lead impactful initiatives to enhance data pipelines and other data operations capabilities while mentoring and onboarding new support data engineers.Create and maintain technical support documentation, including FAQs, troubleshooting steps, support runbooks, and a knowledge base of solutions to recurring issues.Drive operational best practices and shape workflows for routine consumer support, operations monitoring, incident management, and problem management.Administer account access to tools across the technology stack in accordance with SoX governance policies.THE STATSWhat we're looking for in our next teammatePreferred skills:Strong problem-solving skills with the ability to get to the root causes of issues and implement effective solutions.A customer-oriented mindset with a focus on delivering high-quality support services and attention to detail.Minimum 3-5 years of SQL query and query performance tuning experience.Experience with programming languages such as Python, Java, or R.Experience with dimensional data modeling, batch data ingestion, and real-time technologies like Kafka or Kinesis.Understanding of data warehousing and ETL/ELT processing.Experience with cloud databases such as AWS Redshift or Databricks, and Spark on Databricks, AWS, or Azure.Exceptional communication and interpersonal skills.Exposure to orchestration and monitoring tools like Airflow, Datadog, Databand, and Monte Carlo.Familiarity with data visualization tools such as Tableau or Looker.Experience owning and coordinating issues across multiple time zones, bringing continuity of coverage to the Data tribe.Desired skills:Experience with data streaming patterns and ETL/ELT tools like Dbt.Exposure to database design, performance analysis, and data quality metrics.Exposure to Continuous Integration/Continuous Delivery tools like BuildKite or Github Actions.Exposure to Data Science, Machine Learning, and AI.Player BenefitsWe treat our team rightWe offer amazing benefits above and beyond the basics. We have an array of health plans to choose from (some as low as $0 per paycheck) that include programs for fertility and family planning, mental health support, and fitness benefits. We offer generous paid time off (PTO & sick leave), annual bonus and long-term incentive opportunities (based on performance), 401k with up to a 5% match, commuter benefits , pet insurance, and more - check out all our benefits here:FanDuel Total Rewards. *Benefits differ across location, role, and level.FanDuel is an equal opportunities employer and we believe, as one of our principles states, “We are One Team!”. As such, we are committed to equal employment opportunity regardless of race, color, ethnicity, ancestry, religion, creed, sex, national origin, sexual orientation, age, citizenship status, marital status, disability, gender identity, gender expression, veteran status, or another other characteristic protected by state, local or federal law. We believe FanDuel is strongest and best able to compete if all employees feel valued, respected, and included.The applicable salary range for this position is $116,000 - $145,000 USD, which is dependent on a variety of factors including relevant experience, location, business needs and market demand. This role may offer the following benefits: medical, vision, and dental insurance; life insurance; disability insurance; a 401(k) matching program; among other employee benefits. This role may also be eligible for short-term or long-term incentive compensation, including, but not limited to, cash bonuses and stock program participation. This role includes paid personal time off and 14 paid company holidays. FanDuel offers paid sick time in accordance with all applicable state and federal laws.

"""

In [107]:
def sum_text( resume_text,jd_text):
    gemini_input = f"""
    You are an expert skills detector. You need to summarized resume and job description.

    Given the following job description and candidate resume, do the following:
    1. **Focus on candidate's skills, required skills, responsibility and projects**
    2. **Ignore candidate personal information and company information**.
    3. **summarized_resume** and **summarized_jd** MUST be a list of technical skills and domain terminologies.
    Respond only in the exact JSON format shown below. Do not add any text outside the JSON block.

    ---

    **Job Description:**
    {jd_text}

    ---

    **Candidate Resume:**
    {resume_text}

    
    **Sample Output Format**
    ```json
    {{
    "summarized_resume": [],
    "summarized_jd": []
    }}



    ```
    """

    res = ge_model.generate_content(gemini_input, 
            generation_config={
                "temperature": 0.6,
                "top_k": 3,
                "top_p": 0.9,
                "max_output_tokens": 1000,
            },
            
        )

    return json.loads(res.text.replace("```", "").replace("json", "")) 

# s = sum_text(resume_text, jd_text)
# for k, v in s.items():
#     print(k, v)
    

In [108]:
def keyword_matching(resume_keywords, jd_keywords):
    
    
    matching_keywords = set(resume_keywords) & set(jd_keywords)
    
    
    score = len(matching_keywords) / max(len(set(jd_keywords)), len(set(resume_keywords)))  
    return score*2, matching_keywords


In [109]:
def semantic_scoring(resume_text, jd_text):

    resume_text = "passage: " + resume_text
    jd_text = "query: " + jd_text 
    resume_embedding = model.encode(resume_text, convert_to_tensor=True)
    jd_embedding = model.encode(jd_text, convert_to_tensor=True)

    semantic_similarity = util.pytorch_cos_sim(resume_embedding, jd_embedding).item()

    return semantic_similarity

def final_score(resume_text, jd_text):
    llm_s = llm_grading(resume_text, jd_text)
    se_s = semantic_scoring(resume_text, jd_text)
    print(llm_s)
    print(se_s)
    return 0.7*llm_s["overall_match"] + 0.3*se_s

In [110]:
print(final_score(resume_text, jd_text))

{'overall_match': 0.65, 'missing_or_weak_areas': ['Limited experience with cloud databases such as AWS Redshift or Databricks, and Spark on Databricks, AWS, or Azure.', 'Exposure to orchestration and monitoring tools like Airflow, Datadog, Databand, and Monte Carlo is not clearly demonstrated.', 'Experience with data streaming patterns and ETL/ELT tools like Dbt is not explicitly mentioned.', 'Experience owning and coordinating issues across multiple time zones is not evident.'], 'advice_on_improvements': ['Highlight any experience with cloud data platforms (AWS, Azure, GCP) and specific services like Redshift, Databricks, or Spark.', 'Include projects or experiences where you used orchestration tools (e.g., Airflow) or monitoring tools (e.g., Datadog).', 'If you have used data streaming technologies (e.g., Kafka, Kinesis) or ETL/ELT tools (e.g., DBT), add them to your resume with specific examples.', 'Elaborate on any experience coordinating or resolving issues across different time z